In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import matplotlib.pyplot as plt 
import numpy as np
from functools import partial
import json
from glob import glob

import cv2
import os

import pandas as pd
#import nia22
#from nia22.generator import *

In [3]:
def get_converter(field:str):
    if field == "inst":
        keys = ["S", "T", "L", "V", "M"]
    elif field == "condition":
        keys = ["F", "S", "D", "A", "N"]
    elif field == "posture":
        keys = ["S", "D", "P", "L", "F", 
                "C", "H", "E", "T", "U"]
    else:
        raise ValueError(field)

    return partial(_concat, keys=keys)

def _concat(value, keys=None):
    """concatenate strings"""
    code =""
    for s in keys:
        if s in value:
            code += s
        else:
            code += "X"
    return code

# get total file size
def get_total_file_size(wdir, pattern, flist=False):
    all_files = glob(wdir+ pattern)
    all_files.sort()
    nfiles = len(all_files)
    
    print(f"found {nfiles} files")
    
    file_sizes_Bytes = [os.path.getsize(cc) for cc in all_files]
    tsize_inMB = np.sum(file_sizes_Bytes) / 1024 / 1024
    if tsize_inMB > 1024:
        if flist:
            return nfiles, f"{tsize_inMB/1024:.2f}GB", all_files
        else:
            return nfiles, f"{tsize_inMB/1024:.2f}GB"
    else:
        if flist:
            return nfiles, f"{tsize_inMB:.2f}MB", all_files
        else:
            return nfiles, f"{tsize_inMB:.2f}MB"

def change_age(age):
    if age.startswith("1"):
        age = 10
    elif age.startswith("2"):
        age = 20
    elif age.startswith("3"):
        age = 30
    elif age.startswith("4"):
        age = 40
    elif age.startswith("5"):
        age = 50
    elif age.startswith("6"):
        age = 60
    return age

In [9]:
import pandas as pd
import csv, json
from datetime import datetime

import numpy as np

xlsx = pd.read_excel('/home/hoseung/Dropbox/DeepInsight/2022/NIA/metadata/메타데이터.xlsx')

xlsx = list(xlsx.values)

wdir = "./"

In [10]:
concat_inst = get_converter("inst")
concat_condition = get_converter("condition")
concat_posture = get_converter("posture")

In [13]:
subject

'남'

In [14]:
data

array(['김태훈', '남', 1, 'OB001', '27세 (1996년생)', '미착용', '미착용', '453개',
       Timestamp('2022-08-02 00:00:00'), nan], dtype=object)

In [15]:
nas_dir = './NAS/'

### 공통
loc = "S101"
inst = "STLVM"
condition = "FSDAN"
posture = "SDPLFCHETU"
###
for data in xlsx:
    
    clip_list_fn =[]
    
    gender, subject, medical, age, glasses, mask, ver, date, _ = data[1:10]
    date, hour = str(date).split(" ")
    hour = hour[:-3]
    
    ## Variables below are to be read from csv file ##
    username = "hoseung"
    email = "hschoi@dinsight.ai"
    glasses = "no" if glasses == "미착용" else "yes"
    cosmetic= "no"
    mask = "no" if mask == "미착용" else "yes"
    ver  = 453
    rev = str(0)

    wdir = nas_dir + f'{"".join(date.split("-")[1:])}/processing/S1/{subject:03d}/T1/*/'
    
    n_point, tsize_point = get_total_file_size(wdir, "*.txt")
    n_head, tsize_head = get_total_file_size(wdir+"GazeAngle1/", "*.txt")
    n_gaze, tsize_gaze = get_total_file_size(wdir+"GazeAngle2/", "*.txt")
    n_cam, tsize_cam = get_total_file_size(wdir, "*.txt") 
    n_dcam, tsize_d_cam = get_total_file_size(wdir+"DistCam2Face/", "*.txt") # was display pose
    n_disp, tsize_d_disp = get_total_file_size(wdir, "*.txt")
    n_rgb, tsize_rgb, cl_rgb = get_total_file_size(wdir+"RGB/", "*_rgb_*.mp4", flist=True)
    n_ir, tsize_ir, cl_ir = get_total_file_size(wdir+"IR/", "*_ir_*.mp4", flist=True)
    clip_list_fn = cl_rgb + cl_ir
    ######################################

    x = {
        "id": {"subject":subject, 
               "medical":medical},
        "age":change_age(age), 
        "glasses":glasses, 
        "cosmetic":cosmetic, 
        "mask":mask, 
        "gender":gender,
        "ver":ver, 
        "date":date, 
        "loc":loc, 
        "inst":concat_inst(inst), 
        "condition":concat_condition(condition),
        "posture":concat_posture(posture),
        "rev":rev,
        "image": {"format": {"rgb": "mp4","ir": "mp4", "point":"csv", "head":"csv", "gaze":"csv", "cam":"csv",
                             "d_cam":"csv", "d_disp":"csv"},
                    "spec":{"rgb.w":1920, "rgb.h":1080, "ir.w":1920, "ir.h":1080, 
                             "point":"yyyy-mm-dd,hh:mm:ss,x,y",
                             "head":"yyyy-mm-dd,hh:mm:ss,deg,deg,deg",
                             "gaze":"yyyy-mm-dd,hh:mm:ss,deg,deg,deg",
                             "cam":"yyyy-mm-dd,hh:mm:ss,deg,deg,deg",
                             "d_cam":"yyyy-mm-dd,hh:mm:ss,cm",
                             "d_disp":"yyyy-mm-dd,hh:mm:ss,cm"},
                    "total_size":{"rgb":tsize_rgb, "ir":tsize_ir, "point":tsize_point, "head":tsize_head,
                                  "gaze":tsize_gaze, "cam":tsize_cam, "d_cam":tsize_d_cam, "d_disp":tsize_d_disp},
                    "total_count":{"rgb":n_rgb, "ir":n_ir, "point":n_point, "head":n_head, "gaze":n_gaze,
                                   "cam":n_cam, "d_cam":n_dcam, "d_disp":n_disp}
                 },
        "task":{
            "name":"NIA_EYE",
            "created": date + ", " + hour, 
            "updated":"", 
            "username":username, 
            "email":email},
        "filelist":clip_list_fn
    }

    # Dump json file
    fn = f"{subject:03d}.json"
    with open(fn, "w") as f:
        json.dump(x, f, indent=2, ensure_ascii=False)

found 0 files
found 0 files
found 0 files
found 0 files
found 0 files
found 0 files
found 0 files
found 0 files
found 0 files
found 0 files
found 0 files
found 0 files
found 0 files
found 0 files
found 0 files
found 0 files
found 0 files
found 0 files
found 0 files
found 0 files
found 0 files
found 0 files
found 0 files
found 0 files
found 0 files
found 0 files
found 0 files
found 0 files
found 0 files
found 0 files
found 0 files
found 0 files
found 0 files
found 0 files
found 0 files
found 0 files
found 0 files
found 0 files
found 0 files
found 0 files
found 0 files
found 0 files
found 0 files
found 0 files
found 0 files
found 0 files
found 0 files
found 0 files
found 0 files
found 0 files
found 0 files
found 0 files
found 0 files
found 0 files
found 0 files
found 0 files
found 0 files
found 0 files
found 0 files
found 0 files
found 0 files
found 0 files
found 0 files
found 0 files
found 0 files
found 0 files
found 0 files
found 0 files
found 0 files
found 0 files
found 0 files
found 

Eye-tracker, 얼굴 지향각, 시선 각, 카메라-촬영자 거리, 카메라-디스플레이 거리   
각각은 영상의 매 프레임마다 측정값이 제공 됨.  

이번에는 가짜 값을 만들어냄.

In [7]:
def point_to_degree(xpos, ypos, d_cam, d_disp, ):
    """transformation이 너무... 많은데..?? 
    """

In [9]:
# count the number of frames of the video
conditions = ["F", "S", "D", "A", "N"]
postures = ["S", "D", "P", "L", "F", "C", "H", "E", "T", "U"]
display = ["S", "T", "L", "V", "M"]

wdir = "./"

for fn_clip in clip_list:
    file_name = fn_clip.split("/")[-1]
    display, posture, condition, orientation = file_name.split(".mp4")[0].split("_")
    cap = cv2.VideoCapture(fn_clip)
    if not cap.isOpened():
        continue
    nframes = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    #print(display, posture, condition, orientation)
    fake = Fake_generator(wdir, nframes, 1920, 1080, display, posture, condition, orientation)
    dist = 35 # 정하고 하는거 아님
    fake.gen_d_disp(dist)
    fake.gen_d_cam(dist)
    fake.gen_head()
    fake.gen_pose_d()
    fake.gen_gaze_eye_trakcer()
    fake.save_all()

# Save meta data